#  Switching between Scan Models

**Learning how to switch between different scan models and use this strategy to reconstruct large electron datasets.**

---

In [ ]:
import ptypy, os
import ptypy.utils as u

# This will import the EpsicHdf5Loader class
ptypy.load_ptyscan_module("epsic_loader")

# This will import the GPU engines
ptypy.load_gpu_engines("cuda")  

# Root directory of tutorial data
tutorial_data_home = "/global/homes/b/benedikt/workshop_data/"

# Dataset for this tutorial
dataset = "dls_epsic_80kV_graphene/20200130_163208.h5"

# Absolute path to HDF5 file with raw data
path_to_data = os.path.join(tutorial_data_home, dataset)

# Create parameter tree
p = u.Param()

# Set verbose level to info
p.verbose_level = "interactive"

# Scan label
p.run = "dls_epsic_graphene"

# Make sure bigger than nr. of frames 
# because of ePIE
p.frames_per_block = 100000

# Set io settings (no files saved)
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.interaction = u.Param(active=False)

# Live-plotting during the reconstruction
p.io.autoplot = u.Param()
p.io.autoplot.active=True
p.io.autoplot.threaded = False
p.io.autoplot.layout = "jupyter"
p.io.autoplot.interval = 1

# Define the scan model
p.scans = u.Param()
p.scans.scan_00 = u.Param()
p.scans.scan_00.name = 'BlockGradFull'

# Initial illumination (based on simulated optics)
p.scans.scan_00.illumination = u.Param()
p.scans.scan_00.illumination.model = None
p.scans.scan_00.illumination.photons = None
p.scans.scan_00.illumination.aperture = u.Param()
p.scans.scan_00.illumination.aperture.form = "circ"
p.scans.scan_00.illumination.aperture.size = 0.011
p.scans.scan_00.illumination.propagation = u.Param()
p.scans.scan_00.illumination.propagation.focussed = 0.1732
p.scans.scan_00.illumination.propagation.parallel = -1.5e-8
p.scans.scan_00.illumination.diversity = u.Param()
p.scans.scan_00.illumination.diversity.power = 0.1
p.scans.scan_00.illumination.diversity.noise = [0.5,0.2]

# Initial object
p.scans.scan_00.sample = u.Param()
p.scans.scan_00.sample.model = None
p.scans.scan_00.sample.diversity = None
p.scans.scan_00.sample.process = None

# Coherence parameters (modes)
p.scans.scan_00.coherence = u.Param()
p.scans.scan_00.coherence.num_probe_modes = 1
p.scans.scan_00.coherence.num_object_modes = 1

# Data loader
p.scans.scan_00.data = u.Param()
p.scans.scan_00.data.name = 'EpsicHdf5LoaderFast'
p.scans.scan_00.data.orientation = 2

# Read diffraction data
p.scans.scan_00.data.intensities = u.Param()
p.scans.scan_00.data.intensities.file = path_to_data
p.scans.scan_00.data.intensities.key = "data/frames"

# Scan positions
p.scans.scan_00.data.rotation = 60
p.scans.scan_00.data.stepsize = 3.71875e-11
p.scans.scan_00.data.numpos = 255
# p.scans.scan_00.data.positions = u.Param()
# p.scans.scan_00.data.positions.bounding_box = u.Param()
# p.scans.scan_00.data.positions.bounding_box.fast_axis_bounds = [50,200]
# p.scans.scan_00.data.positions.bounding_box.slow_axis_bounds = [50,200]

# Meta information
# electron_data needs to be set to True 
# if energy is given as electron energy in kV
p.scans.scan_00.data.electron_data = True
p.scans.scan_00.data.psize = 220e-6
p.scans.scan_00.data.energy = 80
p.scans.scan_00.data.distance = 0.1732

# Read detector mask
# p.scans.scan_00.data.mask = u.Param()
# p.scans.scan_00.data.mask.file = path_to_data
# p.scans.scan_00.data.mask.key = "data/mask"
# p.scans.scan_00.data.mask.invert = True

# Determine diffraction center from the data
p.scans.scan_00.data.auto_center = True

# Reconstruct using GPU-accelerated ePIE
p.engines = u.Param()
p.engines.engine = u.Param()
p.engines.engine.name = "EPIE_pycuda"
p.engines.engine.numiter = 5
p.engines.engine.numiter_contiguous = 1
p.engines.engine.alpha = 0.9
p.engines.engine.beta = 0.1
p.engines.engine.object_norm_is_global = True
p.engines.engine.probe_support = None
p.engines.engine.probe_fourier_support = None
p.engines.engine.probe_update_start = 0
p.engines.engine.record_local_error = False
#p.engines.engine.fft_lib = "cuda"

# Run reconstruction
P = ptypy.core.Ptycho(p,level=5)

In [ ]:
## Add second script that reads from output
# Maybe this is a good example of using JSON or YAML file